In [3]:
import cv2
import pytesseract
from deep_translator import GoogleTranslator
from langdetect import detect

# Set the Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Mapping of detected languages to Tesseract language codes
lang_map = {
    'hi': 'hin',
    'gu': 'guj',
    'mr': 'mar',
    'bn': 'ben',
    'ta': 'tam',
    'te': 'tel',
    'kn': 'kan',
    'ml': 'mal',
    'pa': 'pan',
    'or': 'ori',
    # Add more mappings as needed
}

def extract_text_from_image(image_path, lang_code='eng'):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding to preprocess the image
    threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    # Perform text extraction
    config = f'--oem 3 --psm 6 -l {lang_code}'
    text = pytesseract.image_to_string(threshold, config=config)
    return text.strip()

def detect_language(text):
    try:
        lang = detect(text)
        return lang_map.get(lang, 'eng')
    except:
        return 'eng'

def translate_text(text, dest_language='en'):
    translator = GoogleTranslator(source='auto', target=dest_language)
    translation = translator.translate(text)
    return translation

def main(image_path):
    # First, try to extract text using a general approach
    initial_text = extract_text_from_image(image_path, 'eng+hin+guj+mar')
    
    # Detect the language of the extracted text
    detected_lang_code = detect_language(initial_text)
    
    # Re-extract text using the detected language for better accuracy
    if detected_lang_code != 'eng':
        extracted_text = extract_text_from_image(image_path, detected_lang_code)
    else:
        extracted_text = initial_text
    
    print(f"Detected Language: {detected_lang_code}")
    print(f"Extracted Text: {extracted_text}")
    
    # Translate the extracted text into English
    translated_text = translate_text(extracted_text, 'en')
    print("Translated Text:", translated_text)

if __name__ == "__main__":
    image_path = "hindi.png"
    main(image_path)

Detected Language: hin
Extracted Text: । लोदीकाल में किले में महल, कुँऐ और
केले और उसके विशाल खजाने पर
दिया गया। बाबर किले नें इव्ाहिस के
का यहीं राज्णमिषेक हुआ। १३३६ में
बचाया था, उस उपकार के बदले हुमाः
'डिलग्रास में हुमायूँ हार गया । पाँच बर्ष
Translated Text: In the Lodhi period, the fort had palaces, wells and forts and its vast treasure was given to Babur. Babur was crowned king here. In return for the favour of saving the fort in 1336, Humayun was defeated in the battle of Dilgras. After five years, he was made the king of the fort.


In [11]:
import cv2
import pytesseract
from deep_translator import GoogleTranslator
from langdetect import detect_langs

# Set the Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use adaptive thresholding to handle different lighting conditions
    threshold = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Perform dilation and erosion to improve character separation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    dilated = cv2.dilate(threshold, kernel, iterations=1)
    processed_image = cv2.erode(dilated, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path, lang_codes='eng+hin'):
    image = preprocess_image(image_path)
    
    # Perform text extraction using Tesseract OCR
    config = f'--oem 3 --psm 6 -l {lang_codes}'
    text = pytesseract.image_to_string(image, config=config)
    return text.strip()

def detect_language(text):
    detected_languages = detect_langs(text)
    return detected_languages

def translate_text(text, source_lang, dest_language='en'):
    if source_lang != 'en':
        translator = GoogleTranslator(source=source_lang, target=dest_language)
        translation = translator.translate(text)
        return translation
    return text  # Return the text unchanged if it's already in English

def process_text_for_translation(extracted_text):
    words = extracted_text.split()  # Split text into words
    translated_text = []
    
    # Detect the language of each word
    for word in words:
        detected_langs = detect_language(word)
        source_lang = detected_langs[0].lang if detected_langs else 'en'
        
        # Translate the word if it's not in English
        translated_word = translate_text(word, source_lang, 'en')
        translated_text.append(translated_word)
    
    return ' '.join(translated_text)

def main(image_path):
    # Step 1: Extract text from image
    extracted_text = extract_text_from_image(image_path, 'eng+hin')
    
    # Step 2: Process text for selective translation (translate only non-English text)
    translated_text = process_text_for_translation(extracted_text)
    
    print("Extracted Text:", extracted_text)
    print("Translated Text:", translated_text)

if __name__ == "__main__":
    image_path = "multilang_image.png"
    main(image_path)


Extracted Text: हिन्दी Hindi
TO
English 3isft
‘Transiation
Translated Text: hindi Hindi BIG English 3isft ‘Transiation


In [21]:
import easyocr

# Initialize the EasyOCR reader with English and Hindi support
reader = easyocr.Reader(['en', 'hi'])

def extract_text_easyocr(image_path):
    result = reader.readtext(image_path, detail=0)
    extracted_text = " ".join(result)
    
    return extracted_text.strip()

# Main function to handle OCR and display text
def main_easyocr(image_path):
    extracted_text = extract_text_easyocr(image_path)
    print("Extracted Text (EasyOCR):", extracted_text)

if __name__ == "__main__":
    image_path = "multilang_image.png"
    main_easyocr(image_path)


Error initializing EasyOCR: name 'corrupt_msg' is not defined
Extracted Text (EasyOCR): Error: EasyOCR reader not initialized.


In [ ]:
from paddleocr import PaddleOCR
import cv2

# Initialize PaddleOCR with Hindi and English language support
ocr = PaddleOCR(use_angle_cls=True, lang='hi')

# Preprocess the image to improve OCR accuracy
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Binarization for better text detection
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Dilation to enhance text features
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    return dilated

# Extract text using PaddleOCR
def extract_text_paddle(image_path):
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Save the processed image (optional step for debugging)
    processed_image_path = 'processed_image.png'
    cv2.imwrite(processed_image_path, processed_image)

    # Run OCR on the processed image
    result = ocr.ocr(processed_image_path)

    # Extract the detected text
    extracted_text = ""
    for line in result:
        for word_info in line:
            extracted_text += word_info[1][0] + " "

    return extracted_text.strip()

# Main function to handle OCR and display text
def main_paddle(image_path):
    extracted_text = extract_text_paddle(image_path)
    print("Extracted Text (PaddleOCR):", extracted_text)

if __name__ == "__main__":
    image_path = "multilang_image.png"
    main_paddle(image_path)
